In [ ]:
%pip install  git+https://github.com/CompVis/taming-transformers.git
!git clone https://github.com/CompVis/taming-transformers.git && cd taming-transformers && python -m pip install -e .
%pip install "omegaconf>=2.0.0" "pytorch-lightning>=1.0.8" einops transformers

import os
os._exit(00)

In [ ]:
#VQGAN ImageNet (f=16), 1024

!curl -L "https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fckpts%2Flast.ckpt&dl=1" >"last.ckpt"
!curl -L "https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1" >"model.yaml"


#boris/vqgan_f16_16384
!curl -L "https://huggingface.co/boris/vqgan_f16_16384/raw/main/config.yaml" > "config_vqgan_minidalle.yaml"
!curl -L "https://huggingface.co/boris/vqgan_f16_16384/resolve/main/model.ckpt" > "model_vqgan_minidalle.ckpt"

In [ ]:
import sys
sys.path.append(".")

# also disable grad to save memory
import torch
torch.set_grad_enabled(False)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from omegaconf import OmegaConf
import taming
from taming.models.vqgan import VQModel

config_path = "model.yaml"

config = OmegaConf.load(config_path)
model=VQModel(**config.model.params)

ckpt_path = "last.ckpt"
sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
model.load_state_dict(sd, strict=False)